# Connect to H2O managed cloud via Token

In [1]:
import h2o_authn 
import getpass 

print(f"Visit https://internal.dedicated.h2o.ai/auth/get-platform-token to get your platform token") 
token_provider = h2o_authn.TokenProvider(
  refresh_token=getpass.getpass('Enter your platform token: '),
  client_id="hac-platform-public", 
  token_endpoint_url="https://auth.internal.dedicated.h2o.ai/auth/realms/hac/protocol/openid-connect/token" 
)

Visit https://internal.dedicated.h2o.ai/auth/get-platform-token to get your platform token


Enter your platform token:  ········


In [2]:
import h2o_engine_manager 
aiem = h2o_engine_manager.login(
  environment="https://internal.dedicated.h2o.ai", 
  token_provider=token_provider
)

## To view all instance

In [3]:
aiem.dai_engine_client.list_engines()

{'engines': [{'annotations': {},
 'api_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-5553',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7fdb21a99ab0>,
 'config': {},
 'cpu': 24,
 'create_time': datetime.datetime(2024, 4, 3, 1, 46, 35, tzinfo=tzutc()),
 'creator': 'users/16a3816b-792a-4899-8fc7-18b276ab9ab7',
 'creator_display_name': 'feng.bai@h2o.ai',
 'current_idle_duration': None,
 'current_running_duration': None,
 'delete_time': None,
 'display_name': 'New DAI Engine 5553',
 'engine_id': 'new-dai-engine-5553',
 'gpu': 1,
 'login_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-5553/oidc/login',
 'max_idle_duration': '4h',
 'max_running_duration': '10h',
 'memory_bytes': '120Gi',
 'name': 'workspaces/default/daiEngines/new-dai-engine-5553',
 'reconciling': False,
 'resume_time': datetime.datetime(2024, 4, 10, 4, 5, 38, tzinfo=tzutc())

## Connect to specific running instance

In [6]:
dai_engine = aiem.dai_engine_client.get_engine("new-dai-engine-9755")

In [7]:
dai_engine.resume()

In [14]:
dai_engine = aiem.dai_engine_client.get_engine("new-dai-engine-9755")

In [15]:
dai_engine.state

<DAIEngineState.STATE_RUNNING: 'STATE_RUNNING'>

## Connect to DAI instance and get the related statistics

In [16]:
# Connect to Driverless AI python Client
dai = dai_engine.connect()

In [9]:
# list all currently logged in users on this instance
users = dai.admin.list_current_users()

In [10]:
# the results are not uniq. It should be a bug. 
users

['feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai',
 'feng.bai@h2o.ai']

In [19]:
# choose only unique user ids. this should be our bug.
uniq_users = set(users)

In [37]:
uniq_users

{'feng.bai@h2o.ai'}

In [17]:
# list all users works fine
users = dai.admin.list_users()

In [18]:
users

['feng.bai@h2o.ai']

In [38]:
# __dict__ is the backdoor for experiments. It has all related info. Most of them are not exposed to available attributes or functions.
for user in uniq_users:
    experiments = dai.admin.list_experiments(username=user)
    print(experiment.__dict__)

{'_client': <class 'driverlessai._core.Client'> https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-9755, '_owner': 'feng.bai@h2o.ai', '_key': 'c3f1fbbe-dc95-11ee-84da-2602aec18499', '_name': 'suhedati', '_all_datasets': [<driverlessai._admin.DatasetProxy object at 0x7fc1f298cac0>, <driverlessai._admin.DatasetProxy object at 0x7fc1f298f400>, <driverlessai._admin.DatasetProxy object at 0x7fc1f298d600>, <driverlessai._admin.DatasetProxy object at 0x7fc1f298dde0>, <driverlessai._admin.DatasetProxy object at 0x7fc1f298e950>], '_datasets': {'train_dataset': <driverlessai._admin.DatasetProxy object at 0x7fc1f298dde0>, 'validation_dataset': None, 'test_dataset': None}, '_raw_info': {'key': 'c3f1fbbe-dc95-11ee-84da-2602aec18499', 'description': 'suhedati', 'parameters': {'dataset': {'key': 'eabf4c54-dc92-11ee-84da-2602aec18499', 'display_name': 'CreditCard-train-cleaned.csv'}, 'resumed_model': {'key': '', 'display_name': ''}, 'target_col': 'DEFAULT_NEXT

In [30]:
help(experiment)

Help on ExperimentProxy in module driverlessai._admin object:

class ExperimentProxy(ServerJobProxy)
 |  ExperimentProxy(client: '_core.Client', owner: str, raw_info: dict) -> None
 |  
 |  A Proxy for admin access for an experiment in the Driverless AI server.
 |  
 |  Method resolution order:
 |      ExperimentProxy
 |      ServerJobProxy
 |      ServerObjectProxy
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client: '_core.Client', owner: str, raw_info: dict) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  creation_timestamp
 |      Creation timestamp of the experiment in seconds since the epoch
 |      (POSIX timestamp).
 |  
 |  datasets
 |       Datasets used for the experiment.
 |      
 |      Returns:
 |          Dictionary of `train_dataset`,`validation_dataset`, and `test

In [44]:
# dataset example:
datasets = dai.admin.list_datasets(username="feng.bai@h2o.ai")
    

/Users/fengbai/opt/anaconda3/envs/py10/lib/python3.10/site-packages/driverlessai/_admin.py:25: UserWarning: 'Admin.list_datasets' is a beta API that is subject to future changes.
  return func(self, *args, **kwargs)


In [45]:
help(datasets[0])

Help on DatasetProxy in module driverlessai._admin object:

class DatasetProxy(ServerObjectProxy)
 |  DatasetProxy(client: '_core.Client', owner: str, raw_info: dict) -> None
 |  
 |  A Proxy for admin access for a dataset in the Driverless AI server.
 |  
 |  Method resolution order:
 |      DatasetProxy
 |      ServerObjectProxy
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client: '_core.Client', owner: str, raw_info: dict) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  columns
 |      Column names of the dataset.
 |  
 |  creation_timestamp
 |      Creation timestamp of the dataset in seconds since the epoch (POSIX timestamp).
 |  
 |  data_source
 |      Original data source of the dataset.
 |  
 |  description
 |      Description of the dataset.
 |  
 |  file_path
 |      Pat

In [54]:
# agina, __dict__ has everything
dataset.__dict__

{'_client': <class 'driverlessai._core.Client'> https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/new-dai-engine-9755,
 '_owner': 'feng.bai@h2o.ai',
 '_key': 'd7064186-dc92-11ee-84da-2602aec18499',
 '_name': 'CreditCard-test-cleaned.csv',
 '_raw_info': {'progress': 1,
  'status': 0,
  'error': '',
  'message': '[4/4] Computed stats for column SEX',
  'aggregation_status': -1,
  'aggregation_error': '',
  'entity': {'key': 'd7064186-dc92-11ee-84da-2602aec18499',
   'name': 'CreditCard-test-cleaned.csv',
   'file_path': 'feng.bai@h2o.ai/d7064186-dc92-11ee-84da-2602aec18499/CreditCard-test-cleaned.csv.1709823372.7451415.bin',
   'file_size': 1383856,
   'bin_file_path': 'feng.bai@h2o.ai/d7064186-dc92-11ee-84da-2602aec18499/CreditCard-test-cleaned.csv.1709823372.7451415.bin',
   'data_source': 'upload',
   'row_count': 6000,
   'column_count': 26,
   'columns': [{'name': 'C1',
     'data_type': 'int',
     'logical_types': [],
     'datetime_format': '',
     's

## Get all experiments from all users

In [46]:

import datetime
for user in uniq_users:
    experiments = dai.admin.list_experiments(username=user)
    for experiment in experiments:
        print("Name : " , experiment.name)
        print("Key : ", experiment.key)
        print("Status : " , experiment.status())
        print("Experiment owner : " , experiment.owner)
        print("Creasion time : " , datetime.datetime.fromtimestamp(experiment.creation_timestamp))
        print("Running duration: ", experiment.run_duration)
        print("**************************")

Name :  1.vulibopa
Key :  c56cacec-fb4d-11ee-af82-c2f0b117ed01
Status :  Complete
Experiment owner :  feng.bai@h2o.ai
Creasion time :  2024-04-15 13:29:53.921436
Running duration:  216.34308409690857
**************************
Name :  vulibopa
Key :  5f2dc9d0-f859-11ee-86ed-badefca748bd
Status :  Complete
Experiment owner :  feng.bai@h2o.ai
Creasion time :  2024-04-11 19:15:22.851856
Running duration:  235.43124794960022
**************************
Name :  1.suhedati
Key :  9de4aad6-f15b-11ee-83e2-6e358110928f
Status :  Complete
Experiment owner :  feng.bai@h2o.ai
Creasion time :  2024-04-02 21:43:48.922101
Running duration:  657.2178838253021
**************************
Name :  rahuwinu
Key :  6ec5c6ce-e619-11ee-ba6b-e6af9811e484
Status :  Complete
Experiment owner :  feng.bai@h2o.ai
Creasion time :  2024-03-19 13:52:20.301615
Running duration:  618.3723320960999
**************************
Name :  suhedati
Key :  c3f1fbbe-dc95-11ee-84da-2602aec18499
Status :  Complete
Experiment owner :

## Get all datasets from all users

In [53]:
import datetime
for user in uniq_users:
    datasets = dai.admin.list_datasets(username=user)
    for dataset in datasets:
        print("Name : " , dataset.name)
        print("Key : " , dataset.key)
        print("Creasion time : " , datetime.datetime.fromtimestamp(dataset.creation_timestamp))    
        print("Dataset owner : " , dataset.owner)
        print("size in mb: ", round(dataset.file_size/1024/1024,2)," mb")
        print("**************************")

Name :  consumer_complaints.csv
Key :  d89327f2-dc93-11ee-84da-2602aec18499
Creasion time :  2024-03-07 10:03:24.691319
Dataset owner :  feng.bai@h2o.ai
size in mb:  191.65  mb
**************************
Name :  airline_sentiment_train_clean.csv
Key :  eebdf0e4-dc92-11ee-84da-2602aec18499
Creasion time :  2024-03-07 09:56:52.384255
Dataset owner :  feng.bai@h2o.ai
size in mb:  1.34  mb
**************************
Name :  airline_sentiment_test_clean.csv
Key :  ed5f312c-dc92-11ee-84da-2602aec18499
Creasion time :  2024-03-07 09:56:50.085461
Dataset owner :  feng.bai@h2o.ai
size in mb:  0.34  mb
**************************
Name :  CreditCard-train-cleaned.csv
Key :  eabf4c54-dc92-11ee-84da-2602aec18499
Creasion time :  2024-03-07 09:56:45.682822
Dataset owner :  feng.bai@h2o.ai
size in mb:  5.25  mb
**************************
Name :  CreditCard-test-cleaned.csv
Key :  d7064186-dc92-11ee-84da-2602aec18499
Creasion time :  2024-03-07 09:56:12.593795
Dataset owner :  feng.bai@h2o.ai
size in m